# Accessing saved models
See documentation: 
- Client API: https://wml-api-pyclient-dev-v4.mybluemix.net/
- REST API V2: https://cloud.ibm.com/apidocs/watson-data-api-cpd
- REST API V4: https://watson-ml-v4-api.mybluemix.net/

## Create a client connection and retrieve some environment information

In [ ]:
import os, json, requests
from watson_machine_learning_client import WatsonMachineLearningAPIClient

credentials_url = os.getenv('RUNTIME_ENV_APSX_URL')
accessToken = os.getenv('USER_ACCESS_TOKEN')

wml_credentials = {
    "instance_id": "openshift",
    "url": credentials_url,
    "token": accessToken,
    "version": "2.5.0"
}

client = WatsonMachineLearningAPIClient(wml_credentials)
print("Client version: " + client.version)

### Gather some environment information

In [ ]:
user_id = os.getenv('USER_ID')
project_id = os.getenv('PROJECT_ID')
project_name = os.getenv('PROJECT_NAME')
service_instance_url = client.service_instance.get_url()

# We hardcode the guid
# mlInstanceGuid = client.service_instance.get_instance_id()
mlInstanceGuid = "99999999-9999-9999-9999-999999999999"

space_name = user_id + '_space'

print("Space name: " + space_name + ", project id: " + project_id)

## Create a WML space for this specific user

In [ ]:
# Set the default project so we can find the saved model
client.set.default_project(project_id)

In [ ]:
# Create a space for the user
metadata = {
    client.spaces.ConfigurationMetaNames.NAME: space_name,
    client.spaces.ConfigurationMetaNames.DESCRIPTION: space_name,
}
spaces_details = client.spaces.store(meta_props=metadata)


In [ ]:
# List the spaces available
client.spaces.list()

## Space setup
- Associate the space to the project
- Promote the model to the deployment space

In [ ]:
# Get the guid for the space
spaces_details = client.spaces.get_details()
mySpaces_details = next(item for item in spaces_details['resources']
                    if item['entity']["name"] == space_name)
space_uid = mySpaces_details['metadata']['guid']
space_href = client.spaces.get_href(mySpaces_details)
print(space_name + " guid: " + space_uid)

### Associate space with project


In [ ]:
import json, requests
url = service_instance_url + "/v4/spaces/" + space_uid

headers = {
    'Authorization': "Bearer " + accessToken,
    'Content-Type': "application/json"
}
data = [
    {
        "op": "add",
        "path": "/tags",
        "value": [
            {
                "value": "dsx-project." + project_id,
                "description": "guid of associated DSX project"
            }
        ]
    }
]
associate_space_response = requests.patch(
    url, 
    json=data,
    headers=headers, 
    verify=False
)
print("Status code: " + str(associate_space_response.status_code))

In [ ]:
# Execute this if the return code is not 200
# print(json.dumps(associate_space_response.json(), indent=4, sort_keys=True))

### Associate project with space

In [ ]:
url = service_instance_url + "/v2/projects/" + project_id

headers = {
    'Authorization': "Bearer " + accessToken,
    'Content-Type': "application/json"
}
metadata = {
    "compute": [
        {
            "name": space_name,
            "guid": mlInstanceGuid,
            "type": "machine_learning",
            "credentials":{
                "url": credentials_url
            },
            "properties":{
                "space_guid": space_uid
            }
        }
    ]
}
associate_project_response = requests.patch(
    url, 
    json=metadata,
    headers=headers, 
    verify=False
)
print("status code: " + str(associate_project_response.status_code))

In [ ]:
# If an error occurs above, we can get the details with this command.
# print(json.dumps(associate_project_response.json(), indent=4, sort_keys=True))

## Promote the model to the space

### Details of saved model
There is only one model available so it is element zero from the model details.

If there were multiple, we could retrieve it by name with code like:
```
model_name = "<model name>"

model_details = client.repository.get_model_details()
myModel_details = next((item for item in model_details['resources']
                        if item['entity']['name'].strip() == model_name),None)
```

In [ ]:
# Get the details for our saved model
model_details = client.repository.get_model_details()
myModel_details = model_details['resources'][0]

model_uid = myModel_details['metadata']['guid']
model_name = myModel_details['entity']['name']
model_href = client.repository.get_model_href(myModel_details)
print(model_name + " guid: " + model_uid)

### Promote using the REST API

In [ ]:
## New promote code
import json, requests

url = service_instance_url + "/api/rest/catalogs/assets/" + model_uid + "/promote" # this is changed to the client side rest api

headers = {
    'Authorization': "Bearer " + accessToken,
    'Content-Type': "application/json"
}

metadata = {
    "spaceId": space_uid,
    "projectId": project_id,
    "spaceName": space_name,
    "projectName": project_name,
    "assetType": "model",
    "mlInstanceGuid": mlInstanceGuid
}

payload = json.dumps(metadata, separators=(',', ':'))
promote_response = requests.post(url, 
                            data=payload,
                            headers=headers, 
                            verify=False
                        )

# print(promote_response.json());
promotedModelHref = promote_response.json()["href"]

promoted_model_response = requests.get(service_instance_url + promotedModelHref,
                            headers=headers, 
                            verify=False
                        )

# print(promoted_model_response.json());
promotedModelGuid = promoted_model_response.json()["metadata"]["asset_id"]
promotedModelHref = promoted_model_response.json()["href"]

In [ ]:
print("service_instance_url: " + service_instance_url)
print("promotedModelGuid: " + promotedModelGuid)
print("promotedModelHref: " + promotedModelHref)

## Deploy the promoted model

In [ ]:
# Deploy using the REST API

url = service_instance_url + "/v4/deployments"
deployment_name = model_name + '_deployed'

# print("Url: " + url)
# print("Space href: " + space_href)
# print("promoted_model_href: " + promotedModelHref)
# print(" ")
 
metadata = {
    "name": deployment_name,
    "online": {},
    "space": { 
        "href": space_href
    },
    "asset": {
        "href": "/v4/models/" + promotedModelGuid + "?space_id=" + space_uid
    }
}

headers = {
    'Authorization': "Bearer " + accessToken,
    'Content-Type': "application/json"
    }

payload = json.dumps(metadata, separators=(',', ':'))
 
deploy_response = requests.post(url, 
                            headers=headers, 
                            data=payload,
                            verify=False
                        )
deployment_details = deploy_response.json()
deployment_uid = client.deployments.get_uid(deployment_details)
# print(json.dumps(deployment_details, indent=4, sort_keys=True))
print("Deployment uid: " + deployment_uid)

In [ ]:
# See that it was deployed
client.set.default_space(space_uid)
client.deployments.list()

## Score some records
We need to extract the field names frmo the model to use in a scoring call.

In [ ]:
# Input field names 
fieldnames = [x['name'] for x in myModel_details['entity']['schemas']['input'][0]['fields']] 

output = "["
for field in fieldnames:
    output += '"' + field + '",'
output += ']'
output

In [ ]:
# Execute the model
scoring_payload = { client.deployments.ScoringMetaNames.INPUT_DATA:
                      [{
                         'fields': fieldnames,
                         'values': [[1, 45970, 'YES', 'Owner Occupier', 11, 7, 1, 579, 0, 9280, 132000, 'L100'],
                                    [2, 43823, 'YES', 'Living with parents/guardian', 14, 1, 2, 2245, 1, 9795, 175000, 'L130']
                                   ]
                        }]
                      }
predictions = client.deployments.score(client.deployments.get_uid(deployment_details), scoring_payload)
predictions

## Remove the deployment
Deployment are running services, if we don't need it, we can remove it.

In [ ]:
client.deployments.delete(client.deployments.get_uid(deployment_details))

## Remove the space
After removing the space, we are where we were at the beginning of the niotebook with a saved model in WML

In [ ]:
# Remove the space
client.spaces.delete(space_uid)